<a href="https://colab.research.google.com/github/GunaTeja777/AML-Project/blob/main/text_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone OpenAI's CLIP repository
!git clone https://github.com/openai/CLIP.git

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Total 256 (delta 0), reused 0 (delta 0), pack-reused 256 (from 1)
Receiving objects: 100% (256/256), 8.93 MiB | 24.18 MiB/s, done.
Resolving deltas: 100% (133/133), done.


In [ ]:
# SYSTEM SETUP: Install libraries and clone the repo
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q omegaconf pytorch-lightning einops torchmetrics

# Avoid Tensorboard issue
!pip uninstall -y tensorboard tb-nightly
!pip install -q tensorboard==2.13.0

# Clone taming-transformers
!git clone https://github.com/CompVis/taming-transformers.git
%cd taming-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 27.9 MB/s eta 0:00:00
Found existing installation: tensorboard 

In [ ]:
import os
import subprocess

# Create directories (using relative paths from current working directory)
os.makedirs("taming-transformers/models/vqgan_imagenet_f16_16384/configs", exist_ok=True)
os.makedirs("taming-transformers/models/vqgan_imagenet_f16_16384/checkpoints", exist_ok=True)

import urllib.request

# Download config (.yaml)
config_url = "https://raw.githubusercontent.com/CompVis/taming-transformers/master/configs/imagenet_vqgan.yaml"
config_path = "taming-transformers/models/vqgan_imagenet_f16_16384/configs/vqgan_imagenet_f16_16384.yaml"
print("Downloading config file...")
urllib.request.urlretrieve(config_url, config_path)
print(f"Config downloaded to: {config_path}")

# Download checkpoint (.ckpt)
ckpt_url = "https://huggingface.co/valhalla/vqgan_imagenet_f16_16384/resolve/main/pytorch_model.bin"
ckpt_path = "taming-transformers/models/vqgan_imagenet_f16_16384/checkpoints/vqgan_imagenet_f16_16384.ckpt"
print("Downloading checkpoint... This is a large file (~1GB), please wait...")
urllib.request.urlretrieve(ckpt_url, ckpt_path)
print(f"Checkpoint downloaded to: {ckpt_path}")

# Verify files were downloaded
print(f"\nConfig file size: {os.path.getsize(config_path)} bytes")
print(f"Checkpoint file size: {os.path.getsize(ckpt_path)} bytes")

Config downloaded to: taming-transformers/models/vqgan_imagenet_f16_16384/configs/vqgan_imagenet_f16_16384.yaml
Checkpoint downloaded to: taming-transformers/models/vqgan_imagenet_f16_16384/checkpoints/vqgan_imagenet_f16_16384.ckpt

Config file size: 949 bytes
Checkpoint file size: 288677011 bytes


In [ ]:
# Patch taming-transformers to remove deprecated torch._six (Windows compatible)
import os

utils_file = "taming/data/utils.py"
if os.path.exists(utils_file):
    # Read the file
    with open(utils_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Remove the problematic import line
    content = content.replace('from torch._six import string_classes', '')

    # Write the file back
    with open(utils_file, 'w', encoding='utf-8') as f:
        f.write(content)

    print("Successfully patched taming/data/utils.py - removed torch._six import")
else:
    print(f"Warning: {utils_file} not found. Make sure you're in the taming-transformers directory.")


Successfully patched taming/data/utils.py - removed torch._six import


In [ ]:
!pip install ftfy regex tqdm
!pip install einops
!pip uninstall torchtext -y

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch,torchvision,os,pdb,math,imageio
import torchvision.transforms as t
import torchvision.transforms.functional as tf
from torch import nn
import PIL
import matplotlib.pyplot as plt

import yaml
import omegaconf as OmegaConf


import warnings
warnings.filterwarnings("ignore")

In [ ]:
#helper functions
def tensor_to_image(image):
  img=image.clone()
  img=img.mul(255).byte()
  img=img.cpu().numpy().transpose(1,2,0)


  plt.figure(figsize=(10,7))
  plt.axis("off")
  plt.imshow(img)
  plt.show()


def norm(data):
  return (data.clip(-1,1)+1)/2  #it makes from 0 to 1 range

#parameters
lr=.5
wd=.1
batch_size=1
noise_factor=300
total_itr=300
im_shape=[225,400,1] #height ,width,channel
firstsize,secondsize,thirdsize=im_shape
device='cuda'


In [ ]:
## instilizing clip
%cd /content/CLIP
import clip
clipmodel, preprocess = clip.load("ViT-B/32", jit=False)
clipmodel.eval()
print("clip model available models :",clip.available_models())
print("clip model input visual resolution :",clipmodel.visual.input_resolution)

/content/CLIP


100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 50.1MiB/s]


clip model available models : ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
clip model input visual resolution : 224


In [ ]:
#instilizing VQgan
%cd /content/taming-transformers
from taming.models.vqgan import VQModel
from omegaconf import OmegaConf # Import OmegaConf class
import torch
import yaml

def load_config(config_path,display=False):
  config=OmegaConf.load(config_path) # Use OmegaConf.load()
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_model(config,ckpt_path=None):
  model=VQModel(**config.model.params) # Access params under 'model' key
  if ckpt_path is not None:
    sd=torch.load(ckpt_path,map_location='cpu')
    missing,unecpted=model.load_state_dict(sd,strict=False)
  return model.eval()

def generator(x):
  # Need to define 'taming' or pass the model object
  x=taming_model.post_quant_conv(x)
  x=taming_model.decoder(x)
  return x

taming_config = load_config(
    r"/content/taming-transformers/taming-transformers/models/vqgan_imagenet_f16_16384/configs/vqgan_imagenet_f16_16384.yaml",
    display=True
)
taming_model = load_model(
    taming_config,
    r"/content/taming-transformers/taming-transformers/models/vqgan_imagenet_f16_16384/checkpoints/vqgan_imagenet_f16_16384.ckpt"
).to("cuda")


/content/taming-transformers
data:
  params:
    batch_size: 12
    num_workers: 24
    train:
      params:
        config:
          size: 256
      target: taming.data.imagenet.ImageNetTrain
    validation:
      params:
        config:
          size: 256
      target: taming.data.imagenet.ImageNetValidation
  target: main.DataModuleFromConfig
model:
  base_learning_rate: 4.5e-06
  params:
    ddconfig:
      attn_resolutions:
      - 16
      ch: 128
      ch_mult:
      - 1
      - 1
      - 2
      - 2
      - 4
      double_z: false
      dropout: 0.0
      in_channels: 3
      num_res_blocks: 2
      out_ch: 3
      resolution: 256
      z_channels: 256
    embed_dim: 256
    lossconfig:
      params:
        codebook_weight: 1.0
        disc_conditional: false
        disc_in_channels: 3
        disc_start: 250001
        disc_weight: 0.8
      target: taming.modules.losses.vqperceptual.VQLPIPSWithDiscriminator
    n_embed: 1024
  target: taming.models.vqgan.VQModel

Working 

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 74.5MB/s]


8.19kB [00:00, 926kB/s]                    


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx